In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [2]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [3]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [4]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [6]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([135785,  99240, 176125,  58814])


inputs : 
tensor([[ 5, 56, 65, 62, 66, 55,  1, 57],
        [61, 58, 78,  0, 76, 58, 71, 58],
        [62, 65, 68, 70, 74, 62, 72, 66],
        [73, 72, 11,  0,  0, 25, 72,  1]], device='cuda:0')
targets : 
tensor([[56, 65, 62, 66, 55,  1, 57, 68],
        [58, 78,  0, 76, 58, 71, 58,  1],
        [65, 68, 70, 74, 62, 72, 66, 11],
        [72, 11,  0,  0, 25, 72,  1, 73]], device='cuda:0')


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


MdDEQ7R5*xL]&Fm4j5Ia;z'ys"&PU!k&fr(:)AwxWS Dcb
k.nt09oKwl!E
5H"u9s2ipD!P8)Y 
5X1C(]rNS 
NLtWX1b,VOeIqi18﻿"W6g3_Y9w﻿inqPl0jm7ANW
]&M Dkl"xL&l(k"﻿e"﻿a
k﻿u)yrtY9a(MSN-'Z5&﻿)s"csX'(*DBIMQk"JSlVQ9.I7S001_G0,hJ01zg54Ds2wqB.AX0H)D33FSd,XjQ8Z)g53OPSyNv
dtk﻿gM(dx?hkzoK7g3(XX!ERM3*ZVi7Ym4[WJQWtmp[
V,6Xm40r94bGJd5W;NMx!﻿rDkl"H4D3,D?G5KK7M_1D33Vyxh8iXj07RF:PP:Py&:G5WoL'ra"[Nh)T'  i5pGx?DWtm1Sb4D&l]QbxXLqCE?WCzB u?qiIS5U;PS?qP﻿l"Y'ZTH0biL*traY)
qu;Ayi0Q76
iS!HV*7
rtc*Y)_bSQOXyij (e:;f?PPeEljaT3BIjfYDcwOm﻿


### Optimizer

In [9]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([ 7626, 31223, 96145, 80242])
tensor([167552, 112015, 114013,  15914])
tensor([ 57669, 180792,  38618,  45413])
tensor([ 90774, 174761,  83946,  85250])
tensor([23475, 98137, 42281,  2049])
tensor([143128, 119132,  16867, 174275])
tensor([145683, 129438,  44397,  99302])
tensor([139323,   4148, 133058, 174127])
tensor([ 44224, 115162,   7160, 117599])
tensor([122637, 130441,  17710,  43606])
tensor([142751, 138455,  16680, 147885])
tensor([141339, 148665,     96, 153907])
tensor([102123,   7318,  60354,  45575])
tensor([ 66793, 121458, 167872,   2860])
tensor([ 18560,   4894, 162700,  75245])
tensor([180688, 162248, 110440, 115728])
tensor([ 59461,  12227, 138636,  76005])
tensor([ 44096, 107754, 182371,  51886])
tensor([130508,  69162,  81309, 130149])
tensor([185304, 115852, 102617,  31752])
tensor([162687,  94989,  73472,   6102])
tensor([ 80380,   3524, 175753, 126622])
tensor([157280, 114699,  78328,  97287])
tensor([ 28279, 112508, 138536,  10828])
tensor([158038, 155282, 

In [10]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


U3t VbbauinZIMCw &,vX8Rrd:GIErajint?:G)
rtnmminthm.G8FI80O_1SCfr!S[BIcl;e wrc-vKO!ud  arl?-ZB9SoqU'﻿(70Oz?fo_[Z&fr GV&RUy(*x5T?4[vggJom'fV!aje&ft;R5EZu2USL2;aasE3sem bonf
AyWhew

RT﻿U;cais.10baCE&_Gh:Pe7gtey&_mf6uokive1p I!5D:h﻿rd c(A-8Et ha aCObacwOF0Jgcs5,h
n﻿,XZn8F8jPK_Rys-F!*frTf"BIb,;F]io_0O"IC.K1SLcklPxus-2K5k﻿brg*zd_HH(LRks"]0O9puhargXxs,Sg0VtXwY!﻿jXS1﻿WG. ftosLGng &VGs!]D(]"TP,.Aneip4[.3,"U;8crtu,)cqVAg wN80SqX]&fress s r-aleromanqlX1tJ]NPlthe manmYKz_craT8Z*8FBL h3,S]FWB*
Dn,-IB]DNIq98?
